In [1]:
import jpype

In [2]:
jpype.addClassPath("../../java/target/python-matsim-1.0-SNAPSHOT-jar-with-dependencies.jar")
jpype.startJVM(jpype.get_default_jvm_path(), "-Djava.class.path=%s" % jpype.getClassPath())

In [3]:
import sys
sys.path.append('..')
import pythonmatsim.api.events

In [4]:
import pythonmatsim.api.bindings as matsim

In [7]:
import jpype.imports as jimport

In [8]:
jimport.registerDomain('org')

In [9]:
from org.matsim.core.scenario import ScenarioUtils
from org.matsim.core.config import ConfigUtils
from org.matsim.api.core.v01 import Id
from org.matsim.core.controler import Controler
from org.matsim.core.controler.OutputDirectoryHierarchy import OverwriteFileSetting
from org.matsim.api.core.v01.events.handler import LinkEnterEventHandler, LinkLeaveEventHandler

AttributeError: module 'org.matsim' has no attribute 'core'